In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms


import utils
import random
import numpy as np
from tqdm import tqdm

teacher = torchvision.models.vit_b_16(pretrained = True)
student = torchvision.models.vit_b_16(pretrained = True)
# model = torchvision.models.vit_l_32(pretrained=True)

print(f"patch_size : {student.patch_size}")
print(f"image_size : {student.image_size}")
print(f"hidden_dim : {student.hidden_dim}")

use_trained_model = False

# 현재 testset과 trainset의 normalize가 다르다. 그런데 student model의 성능이 teacher model에 비해 월등하다는 것은 domain invariant하다는 것이 아닌가?

normalize = transforms.Normalize(mean=[0.5074,0.4867,0.4411],
                                 std=[0.2011,0.1987,0.2025])

# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
# ])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])


transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
])

dataset_val = torchvision.datasets.CIFAR100(root="./", train=False, transform=transform, download=True)
dataset_train = torchvision.datasets.CIFAR100(root="./", train=True, transform=transform_train, download=True)

val_loader = torch.utils.data.DataLoader(dataset_val,
                                          batch_size=128,
                                          shuffle=True,
                                          num_workers=4)

train_loader = torch.utils.data.DataLoader(dataset_train,
                                          batch_size=128,
                                          shuffle=True,
                                          num_workers=4)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


patch_size : 16
image_size : 224
hidden_dim : 768
Files already downloaded and verified
Files already downloaded and verified


In [2]:
if use_trained_model:
    teacher = torch.load("CIFAR100_vit_b_16_71.pth")
else:
    teacher = utils.VisionTransformer(teacher)
    teacher.heads.head = nn.Linear(768, 100)
                         
student = utils.VisionTransformer(student)
student.heads.head = nn.Linear(768, 100)



S_optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
T_optimizer = optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()

S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)

student = student.cuda()
teacher = teacher.cuda()



In [ ]:


if not use_trained_model:
    criterion_onlylabel = lambda a,b : mse(a*b, b)
    best_acc = 0.0
    stack = 0 
    criterion_CE = nn.CrossEntropyLoss()
    mse = nn.MSELoss()
    softmax = torch.nn.Softmax(dim = 1)
    criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
    criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))




    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(0)


    for epoch in range(100):
        print(f"lr : {S_scheduler.get_last_lr()}")
        T_correct = 0
        S_correct = 0
        all_data = 0

        loss_distill = []
        loss_CE = []
        loss_response = []
        student = student.train()
        for img, label in tqdm(train_loader):
            img = img.cuda()
            label = label.cuda()

            T_optimizer.zero_grad()
            output, _ = teacher(img, 0)

#             output = softmax(output)
            loss = criterion_CE(output, label)
            loss.backward()
            T_optimizer.step()
            
        T_scheduler.step()
        test_acc = utils.test(teacher, val_loader, epoch)


        if test_acc > best_acc:
            stack = 0
            best_acc = test_acc
        else:
            stack+=1
        past_acc = test_acc

        if stack > 5:
            S_scheduler.step()
            stack = 0

        print("=" * 100)        

lr : [0.01]


100%|██████████| 79/79 [00:16<00:00,  4.81it/s]


0 	 test acc : 0.7590000033378601
lr : [0.01]


100%|██████████| 79/79 [00:16<00:00,  4.80it/s]


1 	 test acc : 0.8314999938011169
lr : [0.01]


...

100%|██████████| 79/79 [00:16<00:00,  4.83it/s]


4 	 test acc : 0.8356999754905701
lr : [0.01]


100%|██████████| 79/79 [00:16<00:00,  4.80it/s]


5 	 test acc : 0.8356999754905701
lr : [0.01]


  1%|          | 4/391 [00:02<03:56,  1.63it/s]

In [3]:


if not use_trained_model:
    criterion_onlylabel = lambda a,b : mse(a*b, b)
    best_acc = 0.0
    stack = 0 
    criterion_CE = nn.CrossEntropyLoss()
    mse = nn.MSELoss()
    softmax = torch.nn.Softmax(dim = 1)
    criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
    criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))




    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(0)


    for epoch in range(100):
        print(f"lr : {S_scheduler.get_last_lr()}")
        T_correct = 0
        S_correct = 0
        all_data = 0

        loss_distill = []
        loss_CE = []
        loss_response = []
        student = student.train()
        for img, label in tqdm(train_loader):
            img = img.cuda()
            label = label.cuda()

            T_optimizer.zero_grad()
            output, _ = teacher(img, 0)

#             output = softmax(output)
            loss = criterion_CE(output, label)
            loss.backward()
            T_optimizer.step()
            
        T_scheduler.step()
        test_acc = utils.test(teacher, val_loader, epoch)


        if test_acc > best_acc:
            stack = 0
            best_acc = test_acc
        else:
            stack+=1
        past_acc = test_acc

        if stack > 5:
            S_scheduler.step()
            stack = 0

        print("=" * 100)        

lr : [0.01]


100%|██████████| 79/79 [00:16<00:00,  4.88it/s]


0 	 test acc : 0.5884999632835388
lr : [0.01]


100%|██████████| 79/79 [00:16<00:00,  4.91it/s]


1 	 test acc : 0.7601000070571899
lr : [0.01]


...

97 	 test acc : 0.7771999835968018
lr : [1.0000000000000005e-09]


100%|██████████| 79/79 [00:16<00:00,  4.88it/s]


98 	 test acc : 0.7771999835968018
lr : [1.0000000000000005e-09]


100%|██████████| 79/79 [00:16<00:00,  4.87it/s]

99 	 test acc : 0.7771999835968018


In [4]:
utils.test(teacher, val_loader)

100%|██████████| 79/79 [00:16<00:00,  4.91it/s]

0 	 test acc : 0.7110999822616577


tensor(0.7111, device='cuda:0')

In [5]:
import numpy as np

criterion_onlylabel = lambda a,b : mse(a*b, b)

criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))

    
import torch
import random
import torch.backends.cudnn as cudnn
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
past_acc = 0.0
stack = 0  
for epoch in range(100):
    print(f"lr : {S_scheduler.get_last_lr()}")
    T_correct = 0
    S_correct = 0
    all_data = 0
    
    loss_distill = []
    loss_CE = []
    loss_response = []
    student = student.train()
    for img, label in tqdm(train_loader):
        input_data = img.cuda()
        label = label.cuda()
        
        
        all_data += len(input_data)
        input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=0.9, std = 0.4).cuda()
        
        S_optimizer.zero_grad()
        T_optimizer.zero_grad()

        layer = random.randint(0,  3+len(teacher.encoder.layers))
        input_data, fk = student(input_data,layer)
        input_lrp, fk_lrp = teacher(input_lrp,layer)
        
        distill_loss = mse(fk, fk_lrp)
                    
        CE_loss = criterion_CE(input_data, label)
        
#         response_loss = criterion_response(input_data, input_lrp)
        
        T_correct += sum(label == torch.argmax(input_lrp, dim=1))
        S_correct += sum(label == torch.argmax(input_data, dim=1))
        
        loss_CE.append(CE_loss.item())
        loss_distill.append(distill_loss.item())

        loss = (distill_loss * 2 + CE_loss) / 2
        loss.backward()
        S_optimizer.step()
    print("distill loss : ", sum(loss_distill) / len(loss_distill))
    print("general loss : ", sum(loss_CE) / len(loss_CE))
#     print("response loss : ", sum(loss_response) / len(loss_response))
    
    print(f"Teacher acc: {T_correct / all_data}")
    print(f"Student acc: {S_correct / all_data}")
    test_acc = utils.test(student, val_loader, epoch) # student도 변하는거 확인 완료함
    utils.test(teacher, val_loader, epoch) # student도 변하는거 확인 완료함
    
    if test_acc > past_acc + 0.01:
        stack = 0
    elif past_acc + 0.005 > test_acc:
        stack+=1
    past_acc = test_acc
    
    if stack > 5:
        S_scheduler.step()
        stack = 0
        
    print("=" * 100)

lr : [0.01]


100%|██████████| 391/391 [09:07<00:00,  1.40s/it]


distill loss :  0.2791871545107468
general loss :  1.9853936442938607
Teacher acc: 0.5732799768447876
Student acc: 0.5217800140380859


100%|██████████| 79/79 [00:16<00:00,  4.90it/s]


0 	 test acc : 0.796999990940094


...

100%|██████████| 79/79 [00:16<00:00,  4.88it/s]


81 	 test acc : 0.8837999701499939


100%|██████████| 79/79 [00:16<00:00,  4.88it/s]


81 	 test acc : 0.7110999822616577
lr : [1.0000000000000005e-09]


 29%|██▉       | 113/391 [02:39<06:33,  1.41s/it]

KeyboardInterrupt



In [15]:
test_acc = utils.test(student, val_loader, epoch)

100%|██████████| 79/79 [00:16<00:00,  4.93it/s]

82 	 test acc : 0.8837999701499939


In [14]:
torch.save(student, "models/CIFAR100_VIT_student_8837.pth")